## Diane's stuff 2 230417

In [28]:
from snakemake.io import expand
import pandas as pd
import yaml
import pysam

config_file = 'snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## Diane's stuff

In [1]:
import sys
!$sys.executable -m pip install --user encoded_client

In [ ]:
def get_lr_fastq_ids(output_type='bam'):
    
    def get_genome_bam(experiment_id, cache_dir=None):
        experiment = server.get_json(experiment_id)

        metadata = []
        default_analysis = experiment["default_analysis"]
        for f in experiment["files"]:
            analyses = [x["@id"] for x in f["analyses"]]
            # this check only applies to processed data not reads (default_analysis in analyses and)
            if f["output_type"] in ("reads", "unfiltered alignments"):
                if f["output_type"] == "unfiltered alignments" and not default_analysis in analyses:
                    # skip alignments for older analyses.
                    continue
                reps = f["technical_replicates"]
                assert len(reps) == 1
                reps = reps[0]

                extension = {
                    "fastq": ".fastq.gz",
                    "bam": ".bam",
                }[f["file_format"]]
                target_name = "{}_{}_{}_{}{}".format(experiment["accession"], reps, f["accession"], f["output_type"], extension)
                # print(target_name)
                if cache_dir is None:
                    cache_dir = os.getcwd()
                cache_dir = Path(cache_dir)
                target = cache_dir / target_name

                if not target.exists() or target.lstat().st_size != f["file_size"]:
                    with server.get_response(f["href"], stream=True) as response:
                        with open(target, 'wb') as outstream:
                            shutil.copyfileobj(response.raw, outstream)

                metadata.append({
                    "experiment": experiment["accession"],
                    "description": experiment["description"],
                    "simple_biosample_summary": f["simple_biosample_summary"],
                    "file": f["accession"],
                    "output_type": f["output_type"],
                    "file_size": f["file_size"],
                    "bio_rep": f["biological_replicates"][0],
                    "tech_rep": f["technical_replicates"][0],
                    "target": target,
                })

        return metadata

    server = ENCODED("www.encodeproject.org") 
    cart_url = "https://www.encodeproject.org/carts/829d339c-913c-4773-8001-80130796a367/"    
    
    

In [2]:
from encoded_client.encoded import ENCODED
import os
from pathlib import Path
import pandas
import shutil
from tqdm import tqdm



In [3]:
import encoded_client
# encoded_client.__version__

In [4]:
cart_url = "https://www.encodeproject.org/carts/829d339c-913c-4773-8001-80130796a367/"

In [26]:
def get_genome_bam(experiment_id, cache_dir=None):
    experiment = server.get_json(experiment_id)

    metadata = []
    default_analysis = experiment["default_analysis"]
    for f in experiment["files"]:
        analyses = [x["@id"] for x in f["analyses"]]
        # this check only applies to processed data not reads (default_analysis in analyses and)
        if f["output_type"] in ("reads", "unfiltered alignments"):
            if f["output_type"] == "unfiltered alignments" and not default_analysis in analyses:
                # skip alignments for older analyses.
                continue
            reps = f["technical_replicates"]
            assert len(reps) == 1
            reps = reps[0]
            
            extension = {
                "fastq": ".fastq.gz",
                "bam": ".bam",
            }[f["file_format"]]
            target_name = "{}_{}_{}_{}{}".format(experiment["accession"], reps, f["accession"], f["output_type"], extension)
            # print(target_name)
            if cache_dir is None:
                cache_dir = os.getcwd()
            cache_dir = Path(cache_dir)
            target = cache_dir / target_name
            
            # if not target.exists() or target.lstat().st_size != f["file_size"]:
            #     with server.get_response(f["href"], stream=True) as response:
            #         with open(target, 'wb') as outstream:
            #             shutil.copyfileobj(response.raw, outstream)

            metadata.append({
                "experiment": experiment["accession"],
                "description": experiment["description"],
                "simple_biosample_summary": f["simple_biosample_summary"],
                "file": f["accession"],
                "output_type": f["output_type"],
                "file_size": f["file_size"],
                "bio_rep": f["biological_replicates"][0],
                "tech_rep": f["technical_replicates"][0],
                "target": target,
            })
            
    return metadata

In [27]:
cache_dir = Path("cache")
cache_dir.mkdir(exist_ok=True)

cart = server.get_json(cart_url)

metadata = []
for experiment_id in cart["elements"]:
    metadata.extend(get_genome_bam(experiment_id, cache_dir))

KeyboardInterrupt: 

In [21]:
downloaded_data = pandas.DataFrame(metadata)
downloaded_data

,experiment,description,simple_biosample_summary,file,output_type,file_size,bio_rep,tech_rep,target
0,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF318SKH,reads,255510907,1,1_1,cache/ENCSR293MOX_1_1_ENCFF318SKH_reads.fastq.gz
1,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF430RUG,unfiltered alignments,362638099,1,1_1,cache/ENCSR293MOX_1_1_ENCFF430RUG_unfiltered a...
2,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF211SQY,reads,387217252,1,1_1,cache/ENCSR081NRO_1_1_ENCFF211SQY_reads.fastq.gz
3,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF967OHL,unfiltered alignments,521377683,1,1_1,cache/ENCSR081NRO_1_1_ENCFF967OHL_unfiltered a...
4,ENCSR257JBF,excitatory cortical neurons,male adult (53 years),ENCFF919JFJ,reads,345607718,1,1_1,cache/ENCSR257JBF_1_1_ENCFF919JFJ_reads.fastq.gz
...,...,...,...,...,...,...,...,...,...
271,ENCSR462COR,RUSH brain E3348003,female adult (90 or above years) with Alzheime...,ENCFF565SOM,unfiltered alignments,1674565340,1,1_1,cache/ENCSR462COR_1_1_ENCFF565SOM_unfiltered a...
272,ENCSR257YUB,RUSH brain E4368365,male adult (90 or above years) with Alzheimer'...,ENCFF311CZO,reads,1533309113,1,1_1,cache/ENCSR257YUB_1_1_ENCFF311CZO_reads.fastq.gz
273,ENCSR257YUB,RUSH brain E4368365,male adult (90 or above years) with Alzheimer'...,ENCFF646VGI,unfiltered alignments,1858836988,1,1_1,cache/ENCSR257YUB_1_1_ENCFF646VGI_unfiltered a...
274,ENCSR690QHM,RUSH brain E6341028,female adult (90 or above years) with Alzheime...,ENCFF785KVJ,reads,1160071924,1,1_1,cache/ENCSR690QHM_1_1_ENCFF785KVJ_reads.fastq.gz


In [23]:
len(set(downloaded_data["target"]))

276

In [24]:
downloaded_data.head()

,experiment,description,simple_biosample_summary,file,output_type,file_size,bio_rep,tech_rep,target
0,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF318SKH,reads,255510907,1,1_1,cache/ENCSR293MOX_1_1_ENCFF318SKH_reads.fastq.gz
1,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF430RUG,unfiltered alignments,362638099,1,1_1,cache/ENCSR293MOX_1_1_ENCFF430RUG_unfiltered a...
2,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF211SQY,reads,387217252,1,1_1,cache/ENCSR081NRO_1_1_ENCFF211SQY_reads.fastq.gz
3,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF967OHL,unfiltered alignments,521377683,1,1_1,cache/ENCSR081NRO_1_1_ENCFF967OHL_unfiltered a...
4,ENCSR257JBF,excitatory cortical neurons,male adult (53 years),ENCFF919JFJ,reads,345607718,1,1_1,cache/ENCSR257JBF_1_1_ENCFF919JFJ_reads.fastq.gz


## 230421

In [1]:
from snakemake.io import expand
import pandas as pd
import yaml

In [2]:
config_file = 'snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [5]:
events = ['A3', 'A5', 'AF', 'AL', 'MX', 'RI', 'SE']
expand(config['data']['suppa']['events'],
                        event=events,
                        species='human')

['data/human/suppa/cerberus.events_A3_strict.ioe',
 'data/human/suppa/cerberus.events_A5_strict.ioe',
 'data/human/suppa/cerberus.events_AF_strict.ioe',
 'data/human/suppa/cerberus.events_AL_strict.ioe',
 'data/human/suppa/cerberus.events_MX_strict.ioe',
 'data/human/suppa/cerberus.events_RI_strict.ioe',
 'data/human/suppa/cerberus.events_SE_strict.ioe']

## 230412

In [3]:
from snakemake.io import expand
import pandas as pd
import yaml

In [4]:
config_file = 'snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [14]:
df = pd.read_csv(expand(config['data']['mirna']['files'],
                     species='human')[0],
            sep='\t',
            header=None)
df.columns = ['link']

# remove first line
df = df.loc[~df.link.str.contains('metadata')]

df['id'] = df.link.str.rsplit('/', expand=True, n=1)[1]
df['id'] = df['id'].str.split('.tsv', expand=True, n=1)[0]

In [15]:
df

,link,id
1,https://www.encodeproject.org/files/ENCFF645YX...,ENCFF645YXV
2,https://www.encodeproject.org/files/ENCFF045TD...,ENCFF045TDA
3,https://www.encodeproject.org/files/ENCFF156WY...,ENCFF156WYO
4,https://www.encodeproject.org/files/ENCFF596SH...,ENCFF596SHW
5,https://www.encodeproject.org/files/ENCFF520TS...,ENCFF520TSX
...,...,...
250,https://www.encodeproject.org/files/ENCFF392GL...,ENCFF392GLP
251,https://www.encodeproject.org/files/ENCFF946HW...,ENCFF946HWJ
252,https://www.encodeproject.org/files/ENCFF639FB...,ENCFF639FBD
253,https://www.encodeproject.org/files/ENCFF965LN...,ENCFF965LNA
